Apply your data manipulation and analysis skills to investigate the supply chain of ingredients for making avocado toast in the U.K. Your task is to determine the following information:

The name of the most common country of origin for each of the three key ingredients: avocados, olive oil, and sourdough.
Store the most common country of origin for each ingredient in the following variables: top_avocado_origin, top_olive_oil_origin, and top_sourdough_origin. Ensure that the country names contain only letters (A-Z) and spaces, with no hyphens or other characters.

To focus your analysis, subset each of the DataFrames to include only these relevant columns: 'code', 'lc', 'productnameen', 'quantity', 'servingsize', 'packagingtags', 'brands', 'brandstags', 'categoriestags', 'labelstags', 'countries', 'countriestags', 'origins', 'origins_tags'.

After completing this project, feel free to explore other questions using the food data!

In [19]:
######################################################

Begin by reading the avocado data from CSV file in the data folder - it is actually tab-delimited. This creates quite a large DataFrame, so it's a good idea to subset it to only a smaller number of relevant columns. Then read in the file for relevant category tags for avocados.

Filter avocado data using relevant category tags
Each food DataFrame contains a column called categories_tags, which contains the food item category, e.g., fruits, vegetables, fruit-based oils, etc. Start by dropping rows with null values in categories_tags. This column is comma-separated, so you'll first need to turn it into a column of lists so that you can treat each item in the list as a separate tag. Filter this reduced DataFrame to contain only the rows where there is a relevant category tag.

Your avocado DataFrame should contain a column called origins_tags. Create a variable called top_avocado_origin, containing the top country where avocados in the United Kingdom come from.

The golden rule of programming when performing repetitive tasks such as this one is Don't Repeat Yourself (DRY). Turn the code you created to analyze the avocado data and determine its top country of origin into a general function that can be used to do the same with each of the other ingredients. You should also add new steps in it to handle ties, which wasn't necessary for the avocado data.

Just as you did with the avocado data, create the variables top_olive_oil_origin, and top_sourdough_origin, using the relevant category data and analyzing country origin data. To determine these last two origin variables, you'll call the function you've created.

In [177]:
import pandas as pd
relevant_avocado_categories = pd.read_csv("../data/relevant_avocado_categories.txt")
relevant_olive_oil_categories = pd.read_csv("../data/relevant_olive_oil_categories.txt")
relevant_sourdough_categories = pd.read_csv("../data/relevant_sourdough_categories.txt")
avocado = pd.read_csv("../data/avocado.csv", sep='\t')
olive_oil = pd.read_csv("../data/olive_oil.csv", sep='\t', low_memory=False)
sourdough = pd.read_csv("../data/sourdough.csv", sep='\t')

In [ ]:
# Extract relevant tags
relevant_tags = relevant_avocado_categories["en:avocadoes"].tolist()  # Correct method is 'tolist()'

# Subset to get only the necessary columns
avocado_subset = avocado[['lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags',
                          'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries',
                          'countries_tags', 'origins', 'origins_tags']]

# Drop rows with NA values in 'categories_tags'
avocado_subset_cleaned = avocado_subset.dropna(subset=["categories_tags"])

# Split 'categories_tags' into a list
avocado_subset_cleaned.loc[:, 'categories_tags'] = avocado_subset_cleaned['categories_tags'].str.split(',')

# Expand the lists into separate rows
avocado_subset_cleaned_exploded = avocado_subset_cleaned.explode('categories_tags')

# Filter based on relevant tags
filtered_avocado_subset = avocado_subset_cleaned_exploded[avocado_subset_cleaned_exploded["categories_tags"].isin(relevant_tags)]

# Drop rows with NA values in 'origins_tags'
avocado_cleaned_origin = filtered_avocado_subset.dropna(subset=["origins_tags"])

# Group by 'origins_tags' and count occurrences
avocado_origin_grouped = avocado_cleaned_origin.groupby('origins_tags').size().reset_index(name='count')

# Sort and get the top 1 origin
top_origin = avocado_origin_grouped.sort_values(by="count", ascending=False).head(1)

print(top_origin)

In [163]:
relevant_tags = relevant_avocado_categories["en:avocadoes"].to_list()

In [77]:
# subset to get only column needed
avocado_subset = avocado[['lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']]

In [78]:
# drop NA categories tags values
avocado_subset_cleaned = avocado_subset.dropna(subset=["categories_tags"])

In [79]:
avocado_subset_cleaned.loc[:, 'categories_tags'] = avocado_subset_cleaned['categories_tags'].str.split(',')

In [ ]:
avocado_subset_cleaned_exploded = avocado_subset_cleaned.explode('categories_tags')

In [85]:
avocado_cleaned_origin = filtered_avocado_subset.dropna(subset=["origins_tags"])

In [101]:
avocado_origin_grouped = avocado_cleaned_origin.groupby('origins_tags').size().reset_index(name='count')

In [103]:
top_origin = avocado_origin_grouped.sort_values(by="count", ascending=False).head(1)

In [178]:
import pandas as pd

def top_origin(data: pd.DataFrame, relevant_tags: pd.DataFrame, tag_column: str, origin_column: str) -> pd.DataFrame:
    """
    Filter a DataFrame based on relevant tags and find the top origin.

    Args:
        data (pd.DataFrame): The main DataFrame containing product information.
        relevant_tags (pd.DataFrame): The DataFrame containing relevant tags to filter by.
        tag_column (str): The name of the column containing tags in the main DataFrame.
        origin_column (str): The name of the column containing origin information in the main DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing the most common origin.
    """

    # Extract relevant tags
    tags_list = relevant_tags.iloc[:, 0].tolist()

    # Drop rows with NA values in the tag_column
    data_cleaned = data.dropna(subset=[tag_column])

    # Convert the tag_column to lists
    data_cleaned.loc[:, tag_column] = data_cleaned[tag_column].str.split(',')

    # Expand lists into separate rows
    data_exploded = data_cleaned.explode(tag_column)

    # Filter the DataFrame for relevant tags
    filtered_data = data_exploded[data_exploded[tag_column].isin(tags_list)]

    # Drop rows with NA values in the origin_column
    data_cleaned_origin = filtered_data.dropna(subset=[origin_column])

    # Group by origin_column and count occurrences
    origin_grouped = data_cleaned_origin.groupby(origin_column).size().reset_index(name='count')

    # Sort by count in descending order and get the top 1 origin
    top_origin = origin_grouped.sort_values(by="count", ascending=False).head(1)


    return top_origin


In [179]:
top_avocado_origin = top_origin(avocado, relevant_avocado_categories, "categories_tags", "origins_tags")
top_olive_oil_origin = top_origin(olive_oil, relevant_olive_oil_categories, "categories_tags", "origins_tags")
top_sourdough_origin = top_origin(sourdough, relevant_sourdough_categories, "categories_tags", "origins_tags")

In [180]:
print(top_avocado_origin, top_olive_oil_origin, top_sourdough_origin)

   origins_tags  count
14      en:peru     35    origins_tags  count
39     en:spain    223    origins_tags  count
3  en:australia      7
